In [1]:
from selenium import webdriver
import time 
import pandas as pd
import numpy as np 
import re 

from bs4 import BeautifulSoup as bs 
import requests

### Use Selenium to Open URL

In [2]:
driver = webdriver.Chrome("C:\chromedriver_win32\chromedriver.exe")
url = 'https://acis.aphis.edc.usda.gov/ords/f?p=118:205'
view_registrants_button = '/html/body/form/div/div[2]/section[2]/table/tbody/tr/td[1]/div/a'

driver.get(url)
driver.find_element_by_xpath(view_registrants_button).click()

### Use Selenium to Click Table Then Itterate through the pages to add Each Element to a List for 1st DF

In [3]:
view_registrants_button = '/html/body/form/div/div[2]/section[2]/table/tbody/tr/td[1]/div/a'
registrants_xpath= '//*[@id="report_R81923131975772430"]/tbody[2]/tr/td/table'
registrants_next_button = '//*[@id="report_R81923131975772430"]/tbody[3]/tr/td/table/tbody/tr/td[4]/a'

counter = 0
lst = []
lst1 = []

driver.find_element_by_xpath(view_registrants_button).click()

while counter <= 173:
    time.sleep(5)
    driver.find_element_by_xpath(registrants_xpath)
    for b in driver.find_elements_by_xpath(registrants_xpath):
        br = b.find_elements_by_tag_name('b')
        lst.append([b.text.replace('\n', ' ').strip() for b in br])
    for tr in driver.find_elements_by_xpath(registrants_xpath):
        tds = tr.find_elements_by_tag_name('td')
        lst1.append([td.text.replace('\n', ' ').strip() for td in tds])
    driver.find_element_by_xpath(registrants_next_button).click()
    time.sleep(2)
    counter += 1 

### Reshape List into DF 

In [4]:
df = pd.DataFrame(np.array(lst).reshape(-1 ,5))

In [5]:
df2 = pd.DataFrame(np.array(lst1).reshape(-1 ,3))

### Rename Columns

In [6]:
df.columns = [['org_name',
               'customer_no',
               'certificate_no',
              'certificate_status',
              'status_date']]

In [7]:
df2.columns = [['0','1','address']]

In [8]:
df.head(2)

,org_name,customer_no,certificate_no,certificate_status,status_date
0,UTAH STATE UNIVERSITY(),2,87-R-0002,ACTIVE,"Jul 17, 2019"
1,BRIGHAM YOUNG UNIVERSITY(),3,87-R-0003,ACTIVE,"Jul 17, 2019"


In [9]:
addresses = []
for i in range(len(df2)): 
  addresses.append(df2.loc[i, 'address'])

In [10]:
df3 = pd.DataFrame(np.array(addresses).reshape(-1 ,1))

In [11]:
df3 = df3.astype(str)

In [12]:
concat = pd.concat([df,df3], axis = 1)
concat.head()

,"(org_name,)","(customer_no,)","(certificate_no,)","(certificate_status,)","(status_date,)",0
0,UTAH STATE UNIVERSITY(),2,87-R-0002,ACTIVE,"Jul 17, 2019","V.P. FOR RESEARCH/1450 OLD MAIN HILL LOGAN, ..."
1,BRIGHAM YOUNG UNIVERSITY(),3,87-R-0003,ACTIVE,"Jul 17, 2019",OFFICE OF RESEARCH & CREATIVE ACTIVITIES A-285...
2,LDS HOSPITAL(),4,87-R-0004,CANCELLED,"Dec 13, 2007",INTERMOUNTAIN HEALTHCARE OFFICE OF RESEARCH 8T...
3,WEBER STATE UNIVERSITY(),5,87-R-0008,ACTIVE,"Nov 29, 2019","3850 Dixon Parkway Department 1027 OGDEN, UT..."
4,UNIVERSITY OF UTAH(),12,87-R-0001,ACTIVE,"Jul 16, 2019",OFFICE OF COMPARATIVE MEDICINE 75 SOUTH 2000 E...


In [13]:
concat.columns = [['org_name',
               'customer_no',
               'certificate_no',
              'certificate_status',
              'status_date',
                   'address']]

In [14]:
concat.head()

,org_name,customer_no,certificate_no,certificate_status,status_date,address
0,UTAH STATE UNIVERSITY(),2,87-R-0002,ACTIVE,"Jul 17, 2019","V.P. FOR RESEARCH/1450 OLD MAIN HILL LOGAN, ..."
1,BRIGHAM YOUNG UNIVERSITY(),3,87-R-0003,ACTIVE,"Jul 17, 2019",OFFICE OF RESEARCH & CREATIVE ACTIVITIES A-285...
2,LDS HOSPITAL(),4,87-R-0004,CANCELLED,"Dec 13, 2007",INTERMOUNTAIN HEALTHCARE OFFICE OF RESEARCH 8T...
3,WEBER STATE UNIVERSITY(),5,87-R-0008,ACTIVE,"Nov 29, 2019","3850 Dixon Parkway Department 1027 OGDEN, UT..."
4,UNIVERSITY OF UTAH(),12,87-R-0001,ACTIVE,"Jul 16, 2019",OFFICE OF COMPARATIVE MEDICINE 75 SOUTH 2000 E...


In [15]:
concat.to_excel(r'C:\Users\harri\OneDrive\Documents\USDA Reports\Registrants.xlsx', index = False)